In [80]:
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from nltk.tokenize import RegexpTokenizer
import nltk
import re 
import os
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import nltk
from nltk.stem import WordNetLemmatizer
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from textblob import TextBlob
lemmatizer = WordNetLemmatizer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('wordnet')
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
nltk.download('vader_lexicon')

nltk.download('stopwords')
nltk.download('punkt')

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [81]:
df = pd.read_csv('/content/drive/MyDrive/train_data.csv')
df.head()

,artist,song_name,lyrics_processed,link,date,sentiment,Description,Featured Artist,genre,lyrics,explicit_rating,year,decade,new,blob,polarity,subjectivity,blob_score,negative,neutral,positive,compound,SI Score,NLTK Class
0,Adele,Hello,hello it's me i was wondering if after all the...,http://www.youtube.com/watch?v=YQHsXMglC9A,2015-10-23,negative,"In this digital and single-driven age, it’s ve...",NaN,Pop,"[Verse 1]\nHello, it's me\nI was wondering if ...",0.0,2015.0,2010s,hello wondering years like meet go everything ...,"Sentiment(polarity=-0.17681159420289855, subje...",-0.176812,0.513043,negative,0.185,0.712,0.103,-0.6808,neutral,neutral
1,Adele,Someone Like You,i heard that you're settled down that you foun...,http://www.youtube.com/watch?v=hLQl3WQQoQ0,2011-01-24,negative,"In this digital and single-driven age, it’s ve...",NaN,Pop,[Verse 1]\nI heard that you're settled down\nT...,0.0,2011.0,2010s,heard settled found girl married heard dreams ...,"Sentiment(polarity=0.28095238095238095, subjec...",0.280952,0.442857,positive,0.234,0.451,0.315,0.9574,positive,non-neutral
2,Adele,When We Were Young,everybody loves the things you do from the way...,http://www.youtube.com/watch?v=WyVS2N8aK-U,2015-11-20,neutral,"In this digital and single-driven age, it’s ve...",NaN,Pop,[Verse 1]\nEverybody loves the things you do\n...,0.0,2015.0,2010s,everybody loves things way talk way move every...,"Sentiment(polarity=0.062037037037037064, subje...",0.062037,0.417824,positive,0.144,0.538,0.318,0.9867,positive,non-neutral
3,Adele,All I Ask,i will leave my heart at the door i won't say ...,http://www.youtube.com/watch?v=GcHVHPPWkvI,2015-11-20,negative,"In this digital and single-driven age, it’s ve...",NaN,Pop,[Verse 1]\nI will leave my heart at the door\n...,0.0,2015.0,2010s,leave heart door say word said know play prete...,"Sentiment(polarity=-0.19642857142857142, subje...",-0.196429,0.520635,negative,0.228,0.444,0.328,0.9062,positive,non-neutral
4,Adele,Rolling in the Deep,there's a fire starting in my heart reaching a...,http://www.youtube.com/watch?v=rYEDA3JcQqw,2010-11-29,negative,"In this digital and single-driven age, it’s ve...",NaN,Pop,[Verse 1]\nThere's a fire starting in my heart...,1.0,2010.0,2010s,fire starting heart reaching fever pitch bring...,"Sentiment(polarity=0.0859375, subjectivity=0.4...",0.085938,0.449132,positive,0.177,0.542,0.280,0.9926,positive,non-neutral


In [82]:
df['index'] = df.index

Explortory Analysis

In [83]:
df['NLTK Class'].value_counts()

non-neutral    5172
neutral        4829
Name: NLTK Class, dtype: int64

In [84]:
df['sentiment'][4211] = df['sentiment'][4210]
df['sentiment'][4212] = df['sentiment'][4210]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [85]:
df_new = df[(df['blob_score'] == 'neutral')]
df_new = df_new[(df_new['NLTK Class'] == 'non-neutral')]
for i in df_new['song_name']:
  for j in range(len(df)):
    if df['song_name'][j] == i:
      df['blob_score'][j] = df['SI Score'][j]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
df_neu = df[(df['NLTK Class'] == 'neutral')]
len(df_neu)

4829

In [87]:
df_test = df[(df['NLTK Class'] == 'non-neutral')]
len(df_test)

5172

In [88]:
x_test = list(df_test['lyrics_processed'])
y_test = list(df_test['blob_score'])

In [89]:
df_real = df[df['sentiment'].isna()]
len(df_real)

9001

In [90]:
df_real['NLTK Class'].value_counts()

non-neutral    4664
neutral        4337
Name: NLTK Class, dtype: int64

In [160]:
df_evaluation = df[df['sentiment'].notna()]
len(df_evaluation)

1000

In [94]:
df_senti = df_real[(df_real['NLTK Class'] == 'non-neutral')]
df_senti['index'] = df_senti.index
len(df_senti)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


4664

In [95]:
x_train = list(df_senti['lyrics_processed'])
y_train = list(df_senti['blob_score'])

In [71]:
import os
import random
import spacy
from spacy.util import minibatch, compounding
import pandas as pd

eval_list = []


def train_model(
    training_data: list, test_data: list, iterations: int = 10
) -> None:
    # Build pipeline
    nlp = spacy.load("en_core_web_sm")
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.create_pipe(
            "textcat", config={"architecture": "simple_cnn"}
        )
        nlp.add_pipe(textcat, last=True)
    else:
        textcat = nlp.get_pipe("textcat")

    textcat.add_label("pos")
    textcat.add_label("neg")

    # Train only textcat
    training_excluded_pipes = [
        pipe for pipe in nlp.pipe_names if pipe != "textcat"
    ]
    with nlp.disable_pipes(training_excluded_pipes):
        optimizer = nlp.begin_training()
        # Training loop
        print("Beginning training")
        print("Loss\tPrecision\tRecall\tF-score")
        batch_sizes = compounding(
            4.0, 32.0, 1.001
        )  # A generator that yields infinite series of input numbers
        for i in range(iterations):
            print(f"Training iteration {i}")
            loss = {}
            random.shuffle(training_data)
            batches = minibatch(training_data, size=batch_sizes)
            for batch in batches:
                text, labels = zip(*batch)
                nlp.update(text, labels, drop=0.2, sgd=optimizer, losses=loss)
            with textcat.model.use_params(optimizer.averages):
                evaluation_results = evaluate_model(
                    tokenizer=nlp.tokenizer,
                    textcat=textcat,
                    test_data=test_data,
                )
                print(
                    f"{loss['textcat']}\t{evaluation_results['precision']}"
                    f"\t{evaluation_results['recall']}"
                    f"\t{evaluation_results['f-score']}"
                )

    # Save model
    with nlp.use_params(optimizer.averages):
        nlp.to_disk("model_artifacts")


def evaluate_model(tokenizer, textcat, test_data: list) -> dict:
    reviews, labels = zip(*test_data)
    reviews = (tokenizer(review) for review in reviews)
    true_positives = 0
    false_positives = 1e-8  # Can't be 0 because of presence in denominator
    true_negatives = 0
    false_negatives = 1e-8
    for i, review in enumerate(textcat.pipe(reviews)):
        true_label = labels[i]["cats"]
        for predicted_label, score in review.cats.items():
            # Every cats dictionary includes both labels, you can get all
            # the info you need with just the pos label
            if predicted_label == "neg":
                continue
            if score >= 0.5 and true_label["pos"]:
                true_positives += 1
            elif score >= 0.5 and true_label["neg"]:
                false_positives += 1
            elif score < 0.5 and true_label["neg"]:
                true_negatives += 1
            elif score < 0.5 and true_label["pos"]:
                false_negatives += 1
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    if precision + recall == 0:
        f_score = 0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"precision": precision, "recall": recall, "f-score": f_score}


def test_model(input_data):
    #  Load saved trained model
    loaded_model = spacy.load("model_artifacts")
    # Generate prediction
    parsed_text = loaded_model(input_data)
    # Determine prediction to return
    if parsed_text.cats["pos"] > parsed_text.cats["neg"]:
        prediction = "Positive"
        score = parsed_text.cats["pos"]
    else:
        prediction = "Negative"
        score = parsed_text.cats["neg"]
    print(
        # f"Review text: {input_data}\nPredicted sentiment: {prediction}"
        f"\tScore:{score} \nPredicted sentiment: {prediction} "
    )


# pos (0) - opinion, neg (1) - no opinion
def load_training_custom():
  items = []
  for i in range(len(x_train)):
    song = x_train[i]
    sentiment = y_train[i]
    cats_obj = {
        'cats': {
            'neg' : sentiment == 'negative',
            'pos' : sentiment == 'positive'
        }
    }
    this_item = (song, cats_obj)
    items.append(this_item)
  random.shuffle(items)
  train_items = items[0:int(0.8*len(items))]
  validation_items = items[int(0.8*len(items)):len(items)]
  final_items = (train_items, validation_items)
  return final_items


if __name__ == "__main__":
    train, test = load_training_custom()
    print("Training model")
    train_model(train, test)
    df1 = pd.DataFrame(eval_list)
    pd.DataFrame.plot(df1)


Training model
Beginning training
Loss	Precision	Recall	F-score
Training iteration 0
15.261721247202797	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 1
2.573039759921365	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 2
0.8802587319064233	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 3
0.6542133309412748	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 4
0.22279891384823713	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 5
0.13498915523814503	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 6
0.08766252985515166	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 7
0.0694435788464034	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 8
0.06364724278682843	0.7041800643011342	0.9999999999847793	0.8264150943292276
Training iteration 9
0.06440481101162732	0.7041800643011342	

TypeError: ignored

In [77]:
!unzip file.zip

Archive:  file.zip
   creating: content/model_artifacts/
   creating: content/model_artifacts/textcat/
  inflating: content/model_artifacts/textcat/cfg  
  inflating: content/model_artifacts/textcat/model  
   creating: content/model_artifacts/vocab/
  inflating: content/model_artifacts/vocab/strings.json  
  inflating: content/model_artifacts/vocab/lookups.bin  
 extracting: content/model_artifacts/vocab/key2row  
  inflating: content/model_artifacts/vocab/vectors  
  inflating: content/model_artifacts/vocab/lexemes.bin  
   creating: content/model_artifacts/tagger/
  inflating: content/model_artifacts/tagger/tag_map  
  inflating: content/model_artifacts/tagger/cfg  
  inflating: content/model_artifacts/tagger/model  
   creating: content/model_artifacts/ner/
  inflating: content/model_artifacts/ner/cfg  
  inflating: content/model_artifacts/ner/model  
  inflating: content/model_artifacts/ner/moves  
  inflating: content/model_artifacts/tokenizer  
  inflating: content/model_artifac

In [96]:

def test_model_sentiment(x_test):
    spacy_label = []
    #  Load saved trained model
    loaded_model = spacy.load("/content/content/model_artifacts")
    # Generate prediction
    for i in tqdm(range(len(x_test))):
      parsed_text = loaded_model(x_test[i])
      # Determine prediction to return
      if parsed_text.cats["pos"] > parsed_text.cats["neg"]:
          prediction = 1
          score = parsed_text.cats["pos"]
      else:
          prediction = 0
          score = parsed_text.cats["neg"]
      spacy_label.append(prediction)
    # print(
    #     # f"Review text: {input_data}\nPredicted sentiment: {prediction}"
    #     # f"\tScore:{score} \nPredicted sentiment: {prediction} "

    # )
    return spacy_label
  

In [97]:
spacy_final = test_model_sentiment(x_test)
sid = SentimentIntensityAnalyzer()

100%|██████████| 5172/5172 [06:28<00:00, 13.32it/s]


Ensembling

In [98]:
final_preds = []
for i in tqdm(range(len(x_test))):
  song = x_test[i]
  # spacy1 = test_model(song)
  textblob = round((TextBlob(song).sentiment.polarity + 1)/2)
  scores = sid.polarity_scores(song)
  if scores['neg'] > scores['pos']:
    nltk_score = 0
  else:
    nltk_score = 1
  if textblob + spacy_final[i] + nltk_score == 3:
    final_preds.append("positive")
  elif textblob + spacy_final[i] + nltk_score == 2:
    final_preds.append("positive")
  else:
    final_preds.append("negative")


100%|██████████| 5172/5172 [00:34<00:00, 148.59it/s]


In [107]:
from collections import Counter
Counter(final_preds)

Counter({'negative': 1426, 'positive': 3746})

In [108]:
df_test['sentiment'] = final_preds

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [109]:
df_neu['sentiment'] = 'neutral'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
df_final  = df_test.append(df_neu,ignore_index = False)
df_final =  df_final.sort_values(by=['index'])
print(len(df_final))

10001


In [162]:
df_pred_label = pd.DataFrame()
df_pred_label['song_name'] = df_final['song_name']
df_pred_label['sentiment'] = df_final['sentiment']
df_pred_label['index'] = df_final['index']

In [163]:
df_real_label = pd.DataFrame()
df_real_label['song_name'] = df_evaluation['song_name']
df_real_label['sentiment'] = df_evaluation['sentiment']
df_real_label['index'] = df_evaluation['index']


In [164]:
print(len(df_pred_label))
print(len(df_real_label))

10001
1000


In [165]:
df_real_label['sentiment'].value_counts()

neutral     500
positive    250
negative    250
Name: sentiment, dtype: int64

In [166]:
df_acc = df_real_label.merge(df_pred_label, on ='index',how='left')
len(df_acc)

1000

In [173]:
df_acc

,song_name_x,sentiment_x,index,song_name_y,sentiment_y
0,Hello,negative,0,Hello,neutral
1,Someone Like You,positive,1,Someone Like You,positive
2,When We Were Young,positive,2,When We Were Young,positive
3,All I Ask,positive,3,All I Ask,positive
4,Rolling in the Deep,positive,4,Rolling in the Deep,positive
...,...,...,...,...,...
995,Preacher,neutral,6074,Preacher,neutral
996,Burning Bridges,positive,6075,Burning Bridges,positive
997,Truth to Power,neutral,6082,Truth to Power,neutral
998,Oh My My,positive,6091,Oh My My,neutral


Results of Evaluation Set and Machine Results


In [174]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
y_pred = list(df_acc['sentiment_x'])
y_actual = list(df_acc['sentiment_y'])
print('Confusion Matrix: \n',confusion_matrix(y_pred, y_actual))
print('accuracy: ' + str(accuracy_score(y_pred, y_actual)))
print('F1 Score: '+ str(f1_score(y_pred, y_actual, average='macro')))

Confusion Matrix: 
 [[ 85  43  38]
 [ 18 405  66]
 [  7  44 294]]
accuracy: 0.784
F1 Score: 0.7443387914169323


In [126]:
df_final.to_csv('./machine_predictions.csv',index=False)

Submission File


In [183]:
df_new_neu = df_real[df_real['NLTK Class'] == 'neutral']
len(df_new_neu)

4337

In [184]:
spacy_final = test_model_sentiment(x_train)

100%|██████████| 4664/4664 [05:57<00:00, 13.04it/s]


In [185]:
final_preds = []
for i in tqdm(range(len(x_train))):
  song = x_train[i]
  # spacy1 = test_model(song)
  textblob = round((TextBlob(song).sentiment.polarity + 1)/2)
  scores = sid.polarity_scores(song)
  if scores['neg'] > scores['pos']:
    nltk_score = 0
  else:
    nltk_score = 1
  if textblob + spacy_final[i] + nltk_score == 3:
    final_preds.append("positive")
  elif textblob + spacy_final[i] + nltk_score == 2:
    final_preds.append("positive")
  else:
    final_preds.append("negative")


100%|██████████| 4664/4664 [00:31<00:00, 147.92it/s]


In [186]:
df_senti['sentiment'] = final_preds

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [187]:
df_sub  = df_senti.append(df_new_neu,ignore_index = False)
df_sub =  df_sub.sort_values(by=['index'])
print(len(df_sub))

9001


In [188]:
df_sub.to_csv('./new_predictions.csv',index=False)